![](pictures/shopping_propensity.png)

<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
Ноутбук от 30.09.23 
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>   
**_"Data Science Analyst"_  
_МОДЕЛЬ СКЛОННОСТИ КЛИЕНТОВ К ПОКУПКАМ В ФЕДЕРАЛЬНОЙ СЕТИ ПРОДАЖ СПОРТИВНЫХ ТОВАРОВ_**  

Выполнено по технологии CRISP-DM:  
Аргентов Сергей, г.Липецк  
https://argentov.pro
<b>  </b>  
<b>  </b>  

<h1>СОДЕРЖАНИЕ<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Предобработка-данных" data-toc-modified-id="1.-Предобработка-данных-1">1. Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#1.1.-Анализ-источников-данных" data-toc-modified-id="1.1.-Анализ-источников-данных-1.1">1.1. Анализ источников данных</a></span></li><li><span><a href="#1.2.-Устранение-пропусков-в-данных" data-toc-modified-id="1.2.-Устранение-пропусков-в-данных-1.2">1.2. Устранение пропусков в данных</a></span><ul class="toc-item"><li><span><a href="#1.2.1.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;гендерная-принадлежность-потенциальных-покупателей-товара&quot;-есть-признаки-принадлежности-к-полу-в-наименовании-товара" data-toc-modified-id="1.2.1.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;гендерная-принадлежность-потенциальных-покупателей-товара&quot;-есть-признаки-принадлежности-к-полу-в-наименовании-товара-1.2.1">1.2.1. Проверим гипотезу что для товаров с пропуском в признаке "гендерная принадлежность потенциальных покупателей товара" есть признаки принадлежности к полу в наименовании товара</a></span></li><li><span><a href="#1.2.2.-Проверим-гипотезы-о-причинах-пропусков-знаений-в-признаке-&quot;цвет&quot;" data-toc-modified-id="1.2.2.-Проверим-гипотезы-о-причинах-пропусков-знаений-в-признаке-&quot;цвет&quot;-1.2.2">1.2.2. Проверим гипотезы о причинах пропусков знаений в признаке "цвет"</a></span><ul class="toc-item"><li><span><a href="#1.2.2.1.-Проверим-попала-ли-информация-об-уникальныхзначениях-признака-&quot;цвет&quot;-в-наименование-товара-без-признака-&quot;цвет&quot;" data-toc-modified-id="1.2.2.1.-Проверим-попала-ли-информация-об-уникальныхзначениях-признака-&quot;цвет&quot;-в-наименование-товара-без-признака-&quot;цвет&quot;-1.2.2.1">1.2.2.1. Проверим попала ли информация об уникальныхзначениях признака "цвет" в наименование товара без признака "цвет"</a></span></li><li><span><a href="#1.2.2.2.-Проверим-есть-ли-одноимённые-товары-с-наличием-значения-в-признаке-&quot;цвет&quot;-для-товаров-с-пропущенным-значением-в-признаке-&quot;цвет&quot;" data-toc-modified-id="1.2.2.2.-Проверим-есть-ли-одноимённые-товары-с-наличием-значения-в-признаке-&quot;цвет&quot;-для-товаров-с-пропущенным-значением-в-признаке-&quot;цвет&quot;-1.2.2.2">1.2.2.2. Проверим есть ли одноимённые товары с наличием значения в признаке "цвет" для товаров с пропущенным значением в признаке "цвет"</a></span></li><li><span><a href="#1.2.2.3.-Проставим-признак-цвет-для-оставшихся-3,2-тыс.-товаров-при-наличии-в-названии-товара-и-не-определённого-автоматически-признака-&quot;цвет&quot;" data-toc-modified-id="1.2.2.3.-Проставим-признак-цвет-для-оставшихся-3,2-тыс.-товаров-при-наличии-в-названии-товара-и-не-определённого-автоматически-признака-&quot;цвет&quot;-1.2.2.3">1.2.2.3. Проставим признак цвет для оставшихся 3,2 тыс. товаров при наличии в названии товара и не определённого автоматически признака "цвет"</a></span></li><li><span><a href="#1.2.2.4.-Проверим-однородность-значений-признаков-&quot;цвет&quot;-и-исправим-неоднородные-значения" data-toc-modified-id="1.2.2.4.-Проверим-однородность-значений-признаков-&quot;цвет&quot;-и-исправим-неоднородные-значения-1.2.2.4">1.2.2.4. Проверим однородность значений признаков "цвет" и исправим неоднородные значения</a></span></li></ul></li></ul></li><li><span><a href="#1.3.-Обогащение-данных-таблицы-personal_data-из-дополнительного-источника" data-toc-modified-id="1.3.-Обогащение-данных-таблицы-personal_data-из-дополнительного-источника-1.3">1.3. Обогащение данных таблицы personal_data из дополнительного источника</a></span></li></ul></li></ul></div>

In [56]:
# Загрузка биьлиотек
import pandas as pd
import sqlite3 as db

import datetime as dt
import multiprocessing
from multiprocessing import Pool, Manager

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import pprint

# 1. Предобработка данных

## 1.1. Анализ источников данных

In [2]:
# Предоставленная база данных
with db.connect('data/result_data/shop_database.db') as con:
    rows = con.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    dfs = {}
    for row in rows:
        dfs[row[0]] = pd.read_sql(f'SELECT * FROM {row[0]}', con)
        print(f'\n{row[0]}')
        print(dfs[row[0]].info())


purchases
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786260 entries, 0 to 786259
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           786260 non-null  int64  
 1   product      786260 non-null  object 
 2   colour       666736 non-null  object 
 3   cost         786260 non-null  int64  
 4   product_sex  471548 non-null  float64
 5   base_sale    786260 non-null  int64  
 6   dt           786260 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 42.0+ MB
None

personal_data_coeffs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104989 entries, 0 to 104988
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             104989 non-null  int64  
 1   lbt_coef       104989 non-null  float64
 2   ac_coef        104989 non-null  float64
 3   sm_coef        104989 non-null  float64
 4   personal_coef  10

## 1.2. Устранение пропусков в данных

Пропущенные значения есть в таблице `product_sex` (15%) и `color` (40%). Это может быть следствием технической/организационой ошибок при заполнении данных или может отображать следующее:  
- для признака `product_sex` в случае унисекс товаров данные о гендерной принадлежности потенциальных покупателей товара не проставлялась,
- для признака `colour` часть информации могла попасть в наименование товара, часть информации может быть получены из информации по одноимённым товарам, а в случае когда товар не имеет цветовой характеристики (пищевые продукты, лекарственные средствва и т.п. или товары с незафиксированным признаком цвета по техническим или организационным причинам) цвет товара не проставлялся.  
Проверим вышеуказанные гипотизы

### 1.2.1. Проверим гипотезу что для товаров с пропуском в признаке "гендерная принадлежность потенциальных покупателей товара" есть признаки принадлежности к полу в наименовании товара

In [3]:
# Соберём уникальный перечень покупок
df_product_sex = dfs['purchases'].groupby('product').agg({'product_sex': 'max'})
df_product_sex['product'] = df_product_sex.index
df_product_sex.reset_index(drop=True, inplace=True)
print(f'Уникальные значения гендерной принадлежности '
      f'потенциального покупателя: {df_product_sex.product_sex.unique()}\n'
      f'Количесвто уникальных товаров: {len(df_product_sex)}')

Уникальные значения гендерной принадлежности потенциального покупателя: [nan  0.  1.]
Количесвто уникальных товаров: 23145


In [4]:
# Проверим какие покупки и с какими признаками гендерной принадлежности
# имеют в наименовании товара слова 'муж, жен'
def check_product_name_sex(x, fit) -> int:
    if fit in x:
        return 1
    else:
        return 0

df_product_sex['муж'] = df_product_sex['product'].\
                        apply(lambda x:check_product_name_sex(x,'муж'))
df_product_sex['жен'] = df_product_sex['product'].\
                        apply(lambda x: check_product_name_sex(x,'жен'))

In [5]:
# Проверим все ли товары,
# не имеющие гендерной принадлежности потенциального покупателя
# не имеют в названии явного указания на пол 'муж', 'жен'
df_product_sex['унисекс'] = df_product_sex.\
                            apply(lambda x: 1
                                 if x['муж'] == 0 and x['жен'] == 0
                                 else 0,
                                 axis=1)
# Определим количество строк где в названиии продукта нет гендерной принадлежности
# потенциального покупатнлы и не проставлены соответсвующие признаки
len(df_product_sex[(df_product_sex['унисекс'] != 1) & (df_product_sex.product_sex.isna())])

0

**Вывод:** В признаке `product_sex` пропущенные данные с большой вероятностью обозначают что товар пренадлежит к категории `унисекс`. Для возможности использования данного признака в дальнейшем проставим ему в датасете значение 2.0

In [6]:
# Заменим пустые значения признака `product_sex` на значение 2.0
dfs['purchases'].product_sex = dfs['purchases'].product_sex.fillna(2.0)

### 1.2.2. Проверим гипотезы о причинах пропусков знаений в признаке "цвет"

In [7]:
# первоначальный процент строк без заполненного значения признака "цвет"
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

15.2

#### 1.2.2.1. Проверим попала ли информация об уникальныхзначениях признака "цвет" в наименование товара без признака "цвет"

In [8]:
# Создадим множество используемых названий цветов в товарах с проставленным признаком "цвет"
colors = list(dfs['purchases'][~dfs['purchases']['colour'].isna()]['colour'].
              apply(lambda x: x.split('/')))
colors_all = []
for color in colors:
    if isinstance(color, list):
        colors_all.extend(color)
colors_set = set(colors_all)
print(f'Количесвто уникальных цветов: {len(colors_set)}')

Количесвто уникальных цветов: 361


In [9]:
# Функция проверки наличия информации о цвете товара в наименовании товара
def check_name_product(product):
    colors = []
    for word in product.split(' '):
        if word in colors_set:
            colors.append(word)
    if len(colors):
        return '/'.join(colors)

In [10]:
# Получим датасет с отсутствующими значениями в признаке "цвет"
df_non_colors = dfs['purchases'][dfs['purchases']['colour'].isna()]
df_non_colors['colorus_new'] = df_non_colors['product'].apply(lambda x: check_name_product(x))
indexs_color = df_non_colors[~df_non_colors['colorus_new'].isna()]['colorus_new']

/var/folders/5h/1vkpjwr90l19cmzfgsfrcpvm0000gp/T/ipykernel_1896/1099927576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_colors['colorus_new'] = df_non_colors['product'].apply(lambda x: check_name_product(x))


In [11]:
# Добавим полученные цвета в столбец датасета с покупками
for index, new_color in zip(indexs_color.index, indexs_color[indexs_color.index]):
    dfs['purchases'].loc[index, ['colour']] = new_color

In [12]:
# Процент строк без заполненного значения признака "цвет" после первого этапа предобработки данных
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

15.04

**Вывод:** Данным методом удалось добавить только 0.16% значений признака "цвет"

#### 1.2.2.2. Проверим есть ли одноимённые товары с наличием значения в признаке "цвет" для товаров с пропущенным значением в признаке "цвет"

Проставим каждому товару из списка товаров без признака "цвет" наиболее часто встречающееся значение цвета.  
Для экономии времени при расчёте выполним присвоение в мультипроцессорном режиме

In [13]:
# Загрузим функции поиска во всех записях о товаре наиболее часто встречающихся значений признака "цвет"
# для наименований товаров, по которым есть записи с незаполненым признаком "цвет"
# (функции написаны в отдельном модуле для обеспечения требований блиотеки  multiprocessing)
from package.preparation import add_freq_product_color, freq_product_color

In [14]:
# Сформируем датасеты:

# df_1 - датасет в котором среди уникальных наименований товара будем искать такие товары,
#        в которых проставлено наиболее часто встречающееся значение признака "цвет" для этого товара 
df_1 = dfs['purchases'][['product', 'colour']]

# df_2 - датасет в котором сформируем список всех уникальных товаров с пропущенным значение признака "цвет"
df_2 = df_1[df_1['colour'].isna()].groupby('product').agg({'colour': 'max'})
df_2.reset_index(inplace=True)

In [15]:
######   ЗАПУСКАЕМ РАСЧЁТЫ В МНОГОПРОЦЕССОРНОМ РЕЖИМЕ   ######

In [16]:
# 1. Рассчитываем количество ядер процесса, которые можно задействовать для вычислений
#    (уменьшаем количество доступных ядер на 1 ядро, которое оставляем на системные потребности)
n_cpu = multiprocessing.cpu_count() - 1
print('Количество доступных процессоров:', n_cpu)

# 2. Расчитаем минимальный размер обрабатываемых парралельно подмассивов
# (по кратности ядер, которые будем использовать)
val_sub = int(len(df_2)/n_cpu)
print('Размеры подмассивов:', val_sub, '\n')

Количество доступных процессоров: 9
Размеры подмассивов: 1308 



In [17]:
# Ииспользуем признак работы в главном модуле (для обеспечения требований блиотеки  multiprocessing)
if __name__ == '__main__':
    
    # Начнём замер времени
    start_time = dt.datetime.now()
    print('ВРЕМЯ СТАРТА РАСЧЁТА:', start_time, '\n')
    
    # Распарралелим расчёты между доступными ядрами процессора
    # 3. Создаём менеджера парралельных процессов
    #    и его пространство имён для обмена данными между процессами
    with Manager() as mng:
        ns = mng.Namespace()
        ns.dfs = mng.list()
        df = df_1
        df_u = df_2
        print(f'Размеры массивов: {len(df_u)} и {len(df)}\n\n')
        
        # 4. Делим датасет на составляющие,
        #    которые будут рассчитываться в парралельных процессах
        for i in range(n_cpu - 1):
            df = df_1.copy()
            ns.dfs.append((df_u[val_sub*i:val_sub*(i+1)], df.copy()))
            print(f'Подготовлен {i+1} массив размером: {len(ns.dfs[i][0])} для сравнения с {len(ns.dfs[i][1])}')
        ns.dfs.append((df_u[(n_cpu-1)*val_sub:], df.copy()))
        print(f'Подготовлен последний массив размером: {len(ns.dfs[i+1][0])} для сравнения с {len(ns.dfs[i+1][1])}')

        # 5. Запустим пул парралельных процессов вычислений
        #    и результат объединим в единый датафрейм
        with Pool(processes=n_cpu) as p:
            print(f'... запущен параллельный расчёт на {n_cpu} ядрах процессора ...')
            df_non_colours = pd.concat(list(p.map(add_freq_product_color, ns.dfs)), axis=0)

    # Закончим замер времени
    finish_time = dt.datetime.now()
    print('\n\nВРЕМЯ ОКОНЧАНИЯ РАСЧЁТА:', finish_time)
    print('ВРЕМЯ ВЫПОЛНЕНИЯ:', finish_time - start_time, '\n' * 2)

ВРЕМЯ СТАРТА РАСЧЁТА: 2023-10-03 02:23:59.733000 

Размеры массивов: 11775 и 786260


Подготовлен 1 массив размером: 1308 для сравнения с 786260
Подготовлен 2 массив размером: 1308 для сравнения с 786260
Подготовлен 3 массив размером: 1308 для сравнения с 786260
Подготовлен 4 массив размером: 1308 для сравнения с 786260
Подготовлен 5 массив размером: 1308 для сравнения с 786260
Подготовлен 6 массив размером: 1308 для сравнения с 786260
Подготовлен 7 массив размером: 1308 для сравнения с 786260
Подготовлен 8 массив размером: 1308 для сравнения с 786260
Подготовлен последний массив размером: 1311 для сравнения с 786260
... запущен параллельный расчёт на 9 ядрах процессора ...


ВРЕМЯ ОКОНЧАНИЯ РАСЧЁТА: 2023-10-03 02:25:09.967209
ВРЕМЯ ВЫПОЛНЕНИЯ: 0:01:10.234209 




In [18]:
# Сформируем датасет с вновь заполненным признаком "цвет"
df_3 = df_non_colours[~df_non_colours['freq_color'].isna()]

# Создадим словарь восстановленных данных о признаке "цвет"
recovered_color_data = {}
for key, value in zip(list(df_3['product']), list(df_3['freq_color'])):
    recovered_color_data[key] = value

In [19]:
# Функция добавления восстановленных данных в признак "цвет"
def replace_none_1(product, colour, recovered_color_data):
    if pd.isna(colour):
        if product in list(recovered_color_data.keys()):
            return recovered_color_data[product]
        else:
            return None
    else:
        return colour

In [20]:
# Добавим восстановленные записи признака "цвет" в исходный датасет с товарами
dfs['purchases']['colour'] = dfs['purchases'].apply(lambda x:
                                                    replace_none_1(x['product'],
                                                                   x['colour'],
                                                                   recovered_color_data),
                                                    axis=1)

In [21]:
# Процент строк без заполненного значения признака "цвет" после первого этапа предобработки данных
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

0.42

**Вывод:** Данным методом удалось добавить только 14.62% значений признака "цвет" и довести уровень заполненности данного признака до 99,6%

#### 1.2.2.3. Проставим признак цвет для оставшихся 3,2 тыс. товаров при наличии в названии товара и не определённого автоматически признака "цвет"

Создадим словарь цветов и их сочетаний, визуально выявленных в названиях 3,2 тыс. товаров, в которых не удалось проставить признак "цвет" автоматически

In [22]:
colors_manual = {'(бесцветный)': 'бесцветный', '(зеленый': 'зеленый', 'зеленый)': 'зеленый',
                 '(камуфляж)': 'милитари', '(серый': 'серый', 'серый)': 'серый',
                 'белая': 'белый', 'голубой/белый': 'голубой/белый',
                 'горчичный/темно-серый': 'горчичный/темно-серый',
                 'зеленый/морковный': 'зеленый/морковный', 'красный/никель': 'красный/никель',
                 'люминисцентно-желтый': 'люминисцентно-желтый', 'синий/авокадо': 'синий/авокадо',
                 'фиолетовый/белый': 'фиолетовый/белый',
                 'black': 'черный', 'white': 'белый', 'blue': 'синий', 'red': 'красный',
                 'green': 'green', 'gray': 'серый', 'lavender': 'лавандовый',
                 'blue/green': 'синий/зеленый', 'blue/white': 'синий/зеленый',
                 'bright blue': 'светло-синий', 'dark blue': 'темно-синий',
                 'multicolor': 'мультицвет', 'white/gray/lavender': 'белый/серый/лавандовый'}

In [23]:
# Функция добавления в признак "цвет" восстановленных данных из названия товара
def replace_none_2(product, colour, recovered_color_data):
    if pd.isna(colour):
        for word in product.split(' '):
            if word in list(recovered_color_data.keys()):
                return recovered_color_data[word]
            else:
                return None
    else:
        return colour

In [24]:
# Добавим восстановленные записи признака "цвет" в исходный датасет с товарами
dfs['purchases']['colour'] = dfs['purchases'].apply(lambda x:
                                                    replace_none_2(x['product'],
                                                                   x['colour'],
                                                                   colors_manual),
                                                    axis=1)

In [25]:
# Процент строк без заполненного значения признака "цвет" после первого этапа предобработки данных
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

0.42

**Вывод:** Данным методом удалось добавить только 0.03% значений признака "цвет", уровень заполненности данного признака остался  равным 99,6%

#### 1.2.2.4. Проверим однородность значений признаков "цвет" и исправим неоднородные значения

In [26]:
# Функция приведения всех букв признака цвет к нижнему регистру и замены буквы 'ё' на 'е'
def replace_symbol(colour):
    if not pd.isna(colour):
        colour = colour.lower()
        colour = colour.replace('ё', 'е')
        return colour
    else:
        return None

In [27]:
# Приведём все значения признаков цвет к нижнему регистру и заменим все буквы 'ё' на 'е'
dfs['purchases']['colour'] = dfs['purchases']['colour'].apply(lambda x: replace_symbol(x))

In [28]:
# Создадим множество используемых названий цветов в товарах с проставленным признаком "цвет"
colors = list(dfs['purchases'][~dfs['purchases']['colour'].isna()]['colour'].
              apply(lambda x: x.split('/')))
colors_all = []
for color in colors:
    if isinstance(color, list):
        colors_all.extend(color)
colors_set = set(colors_all)
print(f'Количесвто уникальных цветов: {len(colors_set)}')

Количесвто уникальных цветов: 328


In [29]:
# Создадим словарь одородности названий признака "цвет"
uniformity_colours = {
                      'cеребряный': 'серебряный',
                      'антрацит': 'антрацитовый', 'бeлый': 'белый', 'волна': 'волны',
                      'графит': 'графитовый', 'золото': 'золотистый', 'золотой': 'золотистый',
                      'ирис': 'ирисовый', 'камуфляж': 'камуфляжный', 'клетка': 'клетчатый',
                      'коралл': 'коралловый', 'лайм': 'лаймовый', 'леопард': 'леопардовый',
                      'металлик': 'металлический', 'милитари': 'камуфляжный',
                      'мультиколор': 'многоцветный', 'мультицвет': 'многоцветный',
                      'нюд': 'нюдовый', 'персик': 'персиковый', 'полоски': 'полоска',
                      'св.розовый': 'светло-розовый', 'серебро': 'серебряный',
                      'серебристый': 'серебряный', 'сиинй': 'синий', 'синий-белый': 'сине-белый',
                      'хром': 'хромовый', 'цветной': 'многоцветный',
                      'черный-белый': 'черно-белый', 'черный-желтый': 'черно-желтый',
                      'черный-красный': 'черно-красный', 'яблоко': 'яблочный', 
                      }

In [30]:
# Функция замены значения признака "цвет" на однородное по всему датасету
def replace_none_3(colour, uniformity_colours):
    if not pd.isna(colour):
        words_colour = colour.split('/')
        words_uniformity = []
        for word in words_colour:
            if word in list(uniformity_colours.keys()):
                words_uniformity.append(uniformity_colours[word])
            else:
                words_uniformity.append(word)
        return ('/').join(words_uniformity)
    else:
        return None

In [31]:
# Приведём все значения признака "цвет" к однородным
dfs['purchases']['colour'] = dfs['purchases']['colour'].apply(lambda x:
                                                              replace_none_3(x,
                                                                             uniformity_colours))

In [32]:
# Создадим множество используемых названий цветов в товарах с проставленным признаком "цвет"
colors = list(dfs['purchases'][~dfs['purchases']['colour'].isna()]['colour'].
              apply(lambda x: x.split('/')))
colors_all = []
for color in colors:
    if isinstance(color, list):
        colors_all.extend(color)
colors_set = set(colors_all)
print(f'Количесвто уникальных цветов: {len(colors_set)}')

Количесвто уникальных цветов: 299


In [33]:
# Заменим пустые значения признака "цвет" на значение 'неопределенный',
# так как оно уже существует в значениях признака отдельных товаров
dfs['purchases'].colour = dfs['purchases'].colour.fillna('неопределенный')

In [38]:
dfs['purchases'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786260 entries, 0 to 786259
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           786260 non-null  int64  
 1   product      786260 non-null  object 
 2   colour       786260 non-null  object 
 3   cost         786260 non-null  int64  
 4   product_sex  786260 non-null  float64
 5   base_sale    786260 non-null  int64  
 6   dt           786260 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 42.0+ MB


## 1.3. Обогащение данных таблицы personal_data из дополнительного источника

In [34]:
# Прочитаем восстановленные данные
df_recovered  = pd.read_csv('data/result_data/personal_data.csv')
df_recovered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         15748 non-null  int64 
 1   age        15748 non-null  int64 
 2   education  15748 non-null  object
 3   city       15748 non-null  int64 
 4   country    15748 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 615.3+ KB


In [35]:
# Добавим восстановленные данные в общий датафрейм
dfs['personal_data_recovered'] = pd.concat([dfs['personal_data'], df_recovered], axis=0)
dfs['personal_data_recovered'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104989 entries, 0 to 15747
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         104989 non-null  int64  
 1   gender     89241 non-null   float64
 2   age        104989 non-null  int64  
 3   education  104989 non-null  object 
 4   city       104989 non-null  int64  
 5   country    104989 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 5.6+ MB


In [50]:
# Объединим датасеты 'purchases', 'personal_data_coeffs', 'personal_data' для обучения модели,
# прогнозирующей признак "gender" в записях с недостающими значениями такого признака
df_personal = dfs['personal_data_recovered'].set_index('id').\
                                             join(dfs['personal_data_coeffs'].set_index('id'))
df = dfs['purchases'].set_index('id').join(df_personal)
# Перенесём id покупателя в отдельный признак итогового датасета и сбросим индексы
# (это необходимо для дальнейшей работы при выявлении аномальных записей в датасете)
df.reset_index(drop=False, inplace=True)

In [142]:
# Выдели датасет с полноценными данными признака 'gender' обучения модели классификации
df_train_test = df[~df.gender.isna()]
df_train_test.reset_index(inplace=True)
df_train_test.head(3)

,index,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,0,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
1,1,0,Стол Outventure,зеленый,1499,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
2,2,0,Набор Outventure: стол + 4 стула,бежевый,4799,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072


Перед обучением модели целесообразно проверить данные на аномалии и исключить такие данные из обучающего датасета.  
1) При проверки датасета на аномальные записи используем три метода, результаты которых дополним друг другом для повышения качества обучающей выборки.  
2) При проверки данных на аномалии не будем учитывать значения признаков `id`, `lbt_coef`, `ac_coef`, `sm_coef` - так как эти данные уникальны почти для каждого покупателя, а значит не принесут пользы для факторной оценки данных и дальнейшего обучения модели.  
3) При использовании методов попризнакового выявления аномалий не будем учитывать признаки `product_sex`,`base_sale`,`education` - так как они имеют понятные уникальные значения без аномалий (выявлено визуально). Также не будем учитывать признак `gender` - так как это целевая переменная для обуения модели.  

In [145]:
# Определим колонки для попризнакового выявления аномалий
check_columns = ['index', 'product', 'colour', 'cost', 'dt', 'age', 'city', 'country', 'personal_coef']

In [146]:
# Проведём кодирование каждого признака 
df_fit = pd.DataFrame()
for column in check_columns:
    le = LabelEncoder()
    df_fit[column] = le.fit_transform(df_train_test[column])
df_fit.head()

,index,product,colour,cost,dt,age,city,country,personal_coef
0,0,3259,203,1475,7,31,694,16,13
1,1,17773,433,371,37,31,694,16,13
2,2,12187,35,882,37,31,694,16,13
3,3,15582,1456,1256,0,30,627,16,13
4,4,21057,87,907,0,30,627,16,13


In [154]:
df_fit.index = df_fit['index']
df_fit.drop(columns=['index'], axis=1, inplace=True)

In [155]:
# Инициализируем словарь индексов, в котором будем фиксировать:
# метод выявления аномалий; список индексов записей-аномалий
indexs = {'Quantile': []}

In [156]:
# Заполним словарь индексов записей-аномалий, индексами аномалий,
# выявленными квантилями более 3-сигма
for column in df_fit.columns:
    quantile_indexs = list(df_fit[df_fit[column] > df_fit[column].quantile(0.997)].index)
    if len(quantile_indexs) > 0:
        indexs['Quantile'].extend(quantile_indexs)
indexs['Quantile'] = list(set(indexs['Quantile']))

In [157]:
# # Заполним словарь индексов записей-аномалий, индексами аномалий,
#выявленными Локалфактором и Изолесом
lof = LocalOutlierFactor(n_neighbors=693, n_jobs=-1)
ifo = IsolationForest(max_samples=12300, n_jobs=-1, random_state=42)
pred_lof = lof.fit_predict(df_fit)
pred_ifo = ifo.fit_predict(df_fit)
indexs['LocalOutlierFactor'] = list(df_fit[pred_lof == -1].index)
indexs['IsolationForest'] = list(df_fit[pred_ifo == -1].index)

In [158]:
# Сформируемый единый список индексов, полученных всеми методами
indexs_all = []
# Определим долю выбросов, выявленную каждым методом
for method in indexs.keys():
    count_anomaly = len(indexs[method])
    print(f"Методом '{method}' выявлено {count_anomaly} записей-аномалий,", end=' ')
    print(f"что составляет {round(count_anomaly/len(df_train_test) * 100, 2)}% всех записей")
    indexs_all.extend(indexs[method])
indexs_all = list(set(indexs_all))
indexs_all.sort()
print(f'\nИТОГО вывленных записей-аномалий {len(indexs_all)},', end=' ')
print(f'что составляет {round(len(indexs_all)/len(df_train_test) * 100, 2)}% всех записей')

Методом 'Quantile' выявлено 8928 записей-аномалий, что составляет 1.33% всех записей
Методом 'LocalOutlierFactor' выявлено 35630 записей-аномалий, что составляет 5.32% всех записей
Методом 'IsolationForest' выявлено 61030 записей-аномалий, что составляет 9.11% всех записей

ИТОГО вывленных записей-аномалий 82150, что составляет 12.26% всех записей


In [159]:
# Удалим из обучающего датасета записи с признаками аномалий
df_train_test_norm = df_train_test.drop(index=indexs_all)